# SUP ML 3 - PREDICT

#### **Nombres**:
      Albert Fernández
      Julieta Kaplan
      Roger Torruella Vilchez
      Rory Thomson Nelson
      Santiago Ezequiel Velasco
#### **Fecha**: 27/12/2024

# Librerias

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# modelos
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier: el equivalente del Adaline
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

# metricas
import sklearn.metrics as metrics

# interpretabilidad
import shap

# para visualizar mas columnas en un df
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")  # inhabilita warnings

np.random.seed(42)

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
modelo = pd.read_pickle("/content/drive/MyDrive/DSC0924 – Entregable 2 - Fernández-Kaplan-Torruella-Thomson-Velasco/Pickle/modelo.pkl")


In [ ]:
scaler = pd.read_pickle("/content/drive/MyDrive/DSC0924 – Entregable 2 - Fernández-Kaplan-Torruella-Thomson-Velasco/Pickle/scaler.pkl")

In [ ]:
customer_id = pd.read_pickle('/content/drive/MyDrive/DSC0924 – Entregable 2 - Fernández-Kaplan-Torruella-Thomson-Velasco/Pickle/df_pred_Customer_ID.pkl')

In [ ]:
print(f'Importanción del modelo {modelo}, y de la columna customer_id', '\n', customer_id, f'del anterior notebook ')

Importanción del modelo <catboost.core.CatBoostClassifier object at 0x7ae0c0ebfc10>, y de la columna customer_id 
 0       1090001
1       1090002
2       1090003
3       1090004
4       1090005
         ...   
8455    1099982
8456    1099985
8457    1099986
8458    1099987
8459    1099994
Name: Customer_ID, Length: 8460, dtype: int64 del anterior notebook 


# Carga PREDICT dataset

In [ ]:
df_predecir=pd.read_pickle('/content/drive/MyDrive/DSC0924 – Entregable 2 - Fernández-Kaplan-Torruella-Thomson-Velasco/Pickle/ML1_Processing_pred.pkl')

In [ ]:
df_predecir

,HHstatin_C,HHstatin_I,HHstatin_Otros,HHstatin_UNKW,actvsubs,adjmou,adjqty,adjrev,adults,area_AS,area_C,area_CN,area_CST,area_D,area_DMV,area_GL,area_H,area_LA,area_M,area_NE,area_NF,area_NRM,area_NYC,area_O,area_P,area_S,area_SF,area_T,asl_flag,avg3qty,avg3rev,avg6rev,avgmou,avgqty,avgrev,blck_dat,blck_vce,callfwdv,callwait,ccrndmou,change_mou,change_rev,complete,crclscod_A,crclscod_AA,crclscod_BA,crclscod_CA,crclscod_EA,crclscod_Otros,creditcd,custcare,da,datovr,drop_blk,drop_dat,drop_vce,dualband,dwllsize_A,dwllsize_Otros,dwllsize_UNKW,dwlltype_M,dwlltype_S,dwlltype_UNKW,eqpdays,ethnic_G,ethnic_H,ethnic_N,ethnic_Otros,ethnic_S,ethnic_U,forgntvl,hnd_price,hnd_webcap_UNKW,hnd_webcap_WC,hnd_webcap_WCMB,income,infobase_M,iwylis_vce,kid0_2,kid11_15,kid16_17,kid3_5,kid6_10,lor,marital_A,marital_B,marital_M,marital_S,marital_U,models,months,mou,mou_cdat,mou_cvce,mou_opkv,mou_pead,mou_peav,mou_rvce,mouiwylisv,mouowylisv,new_cell,numbcars,opk_dat,opk_vce,ownrent_own,owylis_vce,peak_dat,peak_vce,phones,plcd_dat,prizm_social_one_C,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,prizm_social_one_UNKW,recv_sms,recv_vce,refurb_new_refurb,roam,rv,threeway,totmrc,truck,unan_dat,unan_vce,uniqsubs,vceovr
0,1,0,0,0,1,958.0,393,182.94,1.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52,30,31.0,159.67,65.50,30.49,0.0,10.333333,0.0,0.000000,1.333333,48.250000,1.155000,40.666667,0,1,0,0,0,0,1,1.333333,0.2475,0.000,12.333333,0.000000,2.000000,1,0,0,1,1,0,0,216.0,0,0,0,0,0,1,1,149.98999,0,0,1,3.000000,1,0.333333,0,0,0,0,0,3.0,0,0,1,0,0,1.0,7,136.750000,0.000000,71.746667,22.653333,0.000000,54.500000,5.406667,0.473333,0.713333,0,0.0,0.000000,13.666667,0,1.333333,0.000000,30.000000,1.0,0.000000,1,0,0,0,0,0,0.0,3.333333,0,0.0975,0,0.333333,29.9900,0,0.0,13.333333,1,0.500000
1,0,0,0,1,1,3998.0,1634,305.59,2.459525,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,181,48,51.0,666.33,272.33,50.93,0.0,0.000000,0.0,1.333333,9.000000,-352.750000,-35.847500,66.333333,0,0,0,0,1,0,0,3.666667,0.4950,0.585,9.000000,0.000000,9.000000,1,0,0,1,0,0,1,101.0,0,0,1,0,0,0,0,99.98999,0,0,1,5.492848,0,1.333333,0,0,0,0,0,5.0,0,0,0,0,1,1.0,7,352.750000,1.023333,98.543333,75.283333,0.000000,106.926667,83.666667,14.286667,12.683333,0,0.0,1.000000,43.666667,1,15.666667,0.000000,38.000000,2.0,1.000000,1,0,0,0,0,0,0.0,31.000000,0,1.8000,0,1.333333,24.2700,0,0.0,10.000000,1,8.700000
2,1,0,0,0,2,1772.0,734,219.45,3.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107,30,31.0,253.14,104.86,31.35,0.0,4.666667,0.0,0.333333,6.333333,-86.500000,0.862500,59.333333,1,0,0,0,0,0,1,4.333333,0.0000,0.000,6.666667,0.000000,2.000000,0,1,0,0,0,1,0,262.0,0,0,1,0,0,0,0,129.98999,0,0,1,6.000000,1,19.666667,0,0,0,0,0,13.0,0,0,1,0,0,1.0,9,241.500000,0.000000,143.530000,94.543333,0.000000,76.356667,27.373333,21.736667,3.760000,1,1.0,0.000000,36.000000,1,9.666667,0.000000,48.666667,1.0,0.000000,0,0,0,1,0,0,0.0,26.666667,0,0.0000,0,1.666667,39.9900,1,0.0,20.666667,2,0.000000
3,0,0,0,1,1,18478.0,9018,2131.17,3.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1045,155,278.0,1847.80,901.80,213.12,0.0,5.000000,0.0,8.333333,0.666667,119.000000,0.247500,723.000000,0,1,0,0,0,0,1,0.333333,4.7025,0.000,25.333333,0.000000,20.333333,0,1,0,0,0,1,0,127.0,0,0,0,0,1,0,0,79.98999,0,0,1,6.000000,1,68.666667,0,0,0,0,0,3.0,0,0,0,0,1,1.0,11,2297.000000,0.000000,769.136667,486.420000,0.000000,961.736667,679.020000,150.656667,167.090000,0,1.0,0.000000,346.000000,1,271.000000,0.000000,723.000000,2.0,0.000000,0,0,1,0,0,0,0.0,628.000000,1,0.0000,0,0.000000,149.9900,0,0.0,329.333333,1,0.000000
4,1,0,0,0,1,2601.0,791,608.66,1.000000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,130,143,91.0,371.57,113.00,86.95,0.0,0.666667,0.0,0.666667,48.000000,195.750000,40.015000,115.333333,0,0,0,1,0,0,1,21.666667,0.7425,0.195,6.666667,0.333333,5.666667,1,0,0,1,0,0,1,37.0,0,0,1,0,0,0,0,149.98999,0,0,1,6.000000,1,1.333333,0,0,0,0,0,1.0,0,0,0,1,0,1.0,9,542.250000,0.010000,343.066667,149.096667,0.000000,205.216667,11.243333,2.100000,14.5

# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

In [ ]:
# Al haber hecho el mismo datacleaning en el ML_1, no tenemos que hacerlo ahora.

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [ ]:
# Ya se hizo en el ML_2.

# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

In [ ]:
# se estandarizaran las variables del df a predecir con los parametros calculados en el X_train
df_predecir = scaler.transform(df_predecir)

In [ ]:
df_predecir

array([[ 1.15883529, -0.39550131, -0.35651631, ..., -0.36480318,
        -0.46666846, -0.42710508],
       [-0.86293541, -0.39550131, -0.35651631, ..., -0.45597435,
        -0.46666846, -0.14266888],
       [ 1.15883529, -0.39550131, -0.35651631, ..., -0.1642266 ,
         0.37476711, -0.44444875],
       ...,
       [ 1.15883529, -0.39550131, -0.35651631, ..., -0.09128966,
        -0.46666846, -0.4068708 ],
       [ 1.15883529, -0.39550131, -0.35651631, ...,  0.00899863,
        -0.46666846,  2.59069362],
       [-0.86293541, -0.39550131, -0.35651631, ...,  0.40103467,
        -0.46666846, -0.05421616]])

# PREDICT

* predict() y predict_proba()

In [ ]:
# Creamos un array con las predicciones del modelo (en unos y ceros)
pred = modelo.predict(df_predecir)
print(f' El resultado del la predicción esta en formato array', '\n',  pred)

 El resultado del la predicción esta en formato array 
 [0 0 0 ... 0 1 0]


In [ ]:
# Creamos un array con las predicciones del modelo (en porcentaje en base uno)
pred_proba = modelo.predict_proba(df_predecir)
pred_proba
print(f' El resultado del la probabilidad de la prediccion esta en formato array', '\n',  pred_proba, '\n', 'Este array es el porcentage sin redondeo del que extraemos la prediccion' )

 El resultado del la probabilidad de la prediccion esta en formato array 
 [[0.61014368 0.38985632]
 [0.55522558 0.44477442]
 [0.86557913 0.13442087]
 ...
 [0.58425826 0.41574174]
 [0.43072946 0.56927054]
 [0.60144035 0.39855965]] 
 Este array es el porcentage sin redondeo del que extraemos la prediccion


In [ ]:
# Formateamos las predicciones en probabilidad, redondeando para omitir decimales innecesarios
pred_proba2 = np.round(pred_proba[:,1]*100,1)
print(f' El resultado del la probabilidad de la prediccion esta en formato array', '\n',  pred_proba2, '\n', 'Este array es el porcentage con redondeo del que extraemos la prediccion' )

 El resultado del la probabilidad de la prediccion esta en formato array 
 [39.  44.5 13.4 ... 41.6 56.9 39.9] 
 Este array es el porcentage con redondeo del que extraemos la prediccion


In [ ]:
# Creamos un dataframe con las predicciones y las probabilidades
df_prediccion = pd.DataFrame()
df_prediccion['Customer_ID'] = customer_id
df_prediccion['prediccion'] = pred
df_prediccion['%_proba'] = pred_proba2
print(f'{df_prediccion}','\n','Podemos ver la prediccion de cada cliente junto a la probabilidad de la prediccion')
df_prediccion


      Customer_ID  prediccion  %_proba
0         1090001           0     39.0
1         1090002           0     44.5
2         1090003           0     13.4
3         1090004           0     48.7
4         1090005           0     34.6
...           ...         ...      ...
8455      1099982           1     54.8
8456      1099985           0     20.9
8457      1099986           0     41.6
8458      1099987           1     56.9
8459      1099994           0     39.9

[8460 rows x 3 columns] 
 Podemos ver la prediccion de cada cliente junto a la probabilidad de la prediccion


,Customer_ID,prediccion,%_proba
0,1090001,0,39.0
1,1090002,0,44.5
2,1090003,0,13.4
3,1090004,0,48.7
4,1090005,0,34.6
...,...,...,...
8455,1099982,1,54.8
8456,1099985,0,20.9
8457,1099986,0,41.6
8458,1099987,1,56.9


In [ ]:
df_prediccion['prediccion'].value_counts()

,count
prediccion,
0,6210
1,2250


# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [ ]:
df_prediccion.to_csv("/content/drive/MyDrive/DSC0924 – Entregable 2 - Fernández-Kaplan-Torruella-Thomson-Velasco/OUTPUT FINAL_PREDICCIONES.csv")